# **Modélisation & IA | RandomForestRegressor**


## **Etape 0️⃣ : Chargement du dataset :**

In [1]:
import pandas as pd
df=pd.read_csv('data5.csv')
print(df.columns)

Index(['date', 'city', 'profil_cotier', 'température', 'humidity',
       'vitesse_vent', 'precipitation', 'temp_lag1', 'humid_lag1',
       'vitesse_vent_lag1', 'precip_lag1', 'temp_lag3', 'humid_lag3',
       'vitesse_vent_lag3', 'precip_lag3', 'temp_lag7', 'humid_lag7',
       'vitesse_vent_lag7', 'precip_lag7', 'urgence_active', 'mois', 'Weekend',
       'Jours Fériés', 'saison', 'Vacances Scolaires', 'Ramadan', 'Nouvel An',
       'Indice de Vague de Chaleur', 'Indice de Vague de Froid ',
       'Indice de Pluie Intense', 'Indice de Tempête', 'Indice de sécheresse',
       'boissons fraiches', 'boissons chaudes', 'snacks sucrés',
       'snacks salés', 'produits laitiers frais', 'produits de jardinage',
       'ustensiles jetables', 'crème solaire', 'équipements d urgence',
       'soins hygiene', 'soins hydratants', 'Charbon',
       'produits anti_moustiques'],
      dtype='object')


## **Etape 1️⃣ : Séparation X (features) / y (cibles)**

In [2]:
#variable cible == les produits : 
targets = [
    'boissons fraiches', 'boissons chaudes', 'snacks sucrés',
    'snacks salés', 'produits laitiers frais', 'produits de jardinage',
    'ustensiles jetables', 'crème solaire', 'équipements d urgence',
    'soins hygiene', 'soins hydratants', 'Charbon',
    'produits anti_moustiques'
]
y = df[targets]



In [3]:
#Features (X) :
drop_cols = ['date'] + targets
X = df.drop(columns=drop_cols)


In [4]:
print(type(X))
print(X.shape , y.shape)

<class 'pandas.core.frame.DataFrame'>
(43769, 31) (43769, 13)



---

## **Etape 2️⃣: Préprocessing pour `RandomForestRegressor`**

###  Ce qu’il **faut faire** :

1. **Encodage des variables catégorielles** : 

   * `RandomForest` ne sait pas gérer des chaînes de caractères.
   * → Utilisation de **OneHotEncoding** (si peu de catégories) ou **OrdinalEncoding** (si l’ordre a du sens).

###  Ce qu’il **ne faut pas faire** :

1. **Standardisation / Normalisation (scaling)** : 

   * Pas nécessaire pour les arbres (ni Decision Tree, ni Random Forest, ni Gradient Boosting).
   * Ils ne sont pas sensibles aux échelles car ils font des **coupures (seuils)**, pas des distances.

---




# **Bibliothèques nécessaires**

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder ,OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns


In [6]:
cat_col = ['profil_cotier','saison','city']
# split :
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)
# Pipelines d'encodage
preprocessor = ColumnTransformer(transformers=[
    ('cat_col', OneHotEncoder(handle_unknown='ignore'), cat_col)
    ],
    remainder='passthrough' # garde les colonnes restantes
                                )


In [7]:
# Pipeline complet
rf = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('regressor', RandomForestRegressor(
                
    ))
])

# Entraînement
rf.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_col',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['profil_cotier', 'saison',
                                                   'city'])])),
                ('regressor', RandomForestRegressor())])

In [8]:
y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)


In [9]:
print(f"R² Train :{ r2_score(y_train, y_pred_train):.4f}")
print(f"R² Test  :{ r2_score(y_test, y_pred_test):.4f}")
print(f"MSE Test :,{ mean_squared_error(y_test, y_pred_test):.4f}")

R² Train :0.9766
R² Test  :0.8354
MSE Test :,1.7907


## **Analyse des performances de modèle de Base :**
| Métrique     | Valeur | Interprétation                                                                            |
| ------------ | ------ | ----------------------------------------------------------------------------------------- |
| **R² Train** | 0.976  | Le modèle explique **97%** de la variance sur les données d'entraînement → **très bon** |
| **R² Test**  | 0.835  | Le modèle explique **83%** de la variance sur les données de test → **bon, mais écart** |
| **MSE Test** | 1.79   | Erreur quadratique moyenne → utilisée pour juger la précision des prédictions             |
> **l’écart important entre R² Train (0.97) et R² Test (0.83) indique probablement un overfitting :**
>* **Le modèle s’adapte très bien aux données d’entraînement,**
>* **Mais il perd en généralisation sur les données de test.**


## **Etape 3️⃣ : Réduction du surapprentissage**

# **Automatiser l’optimisation des hyperparamètres avec GridSearchCV**
| Hyperparamètre      | Rôle principal                              | Impact sur le modèle                             |
| ------------------- | ------------------------------------------- | ------------------------------------------------ |
| `n_estimators`      | Nombre total d’arbres                       | Plus -> meilleure stabilité, plus lent           |
| `max_depth`         | Profondeur max des arbres                   | Plus -> plus complexe, risque overfitting        |
| `min_samples_split` | Nb min d’échantillons pour diviser un noeud | Plus -> arbres plus simples, généralisation      |
| `min_samples_leaf`  | Nb min d’échantillons dans une feuille      | Plus -> feuilles plus robustes, moins de bruit   |
| `max_features`      | Nb de features à considérer pour un split   | Plus -> arbres plus corrélés, moins de diversité |


In [ ]:

# Grid ciblé pour réduire l'overfitting
param_grid_anti_overfitting = {
    # 1. CONTRÔLE DE LA PROFONDEUR 
    'max_depth': [5, 10, 15, 20, 25],  # Limiter la profondeur ( # Profondeur maximale des arbres)
    
    # 2. RÉGULARISATION DES NŒUDS  
    'min_samples_split': [10, 20, 50, 100],  # Plus élevé = moins d'overfitting( # Nombre minimum d'échantillons pour diviser un noeud)
    'min_samples_leaf': [5, 10, 20, 30],     # Plus élevé = moins d'overfitting (# Nombre minimum d'échantillons dans une feuille)
    
    # 3. CONTRÔLE DU NOMBRE D'ARBRES
    'n_estimators': [50, 100, 200],  # Moins critique, mais impact sur temps ( # Nombre d'arbres dans la forêt)
    
    # 4. RÉDUCTION DES FEATURES PAR ARBRE
    'max_features': ['sqrt', 'log2', 0.3, 0.5]  # Réduit la variance (# Nombre de features à considérer à chaque split)
}
# Configuration de GridSearchCV
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_anti_overfitting,
    cv=5,                 # validation croisée 5-fold
    n_jobs=-1,            # utilise tous les cœurs CPU disponibles
    scoring='r2',         # métrique d'évaluation : coefficient de détermination R²
    verbose=2             # affiche les détails de la recherche
)
# Lancement de la recherche sur les hyperparamètres avec les données d'entraînement
grid_search.fit(X_train, y_train)

# Affichage du meilleur score obtenu et des meilleurs paramètres
print("Meilleur score R² : {:.4f}".format(grid_search.best_score_))
print("Meilleurs hyperparamètres :", grid_search.best_params_)


| Paramètre           | Valeur optimale | Rôle                                                                                                              |
| ------------------- | --------------- | ----------------------------------------------------------------------------------------------------------------- |
| `n_estimators`      | 200             | Nombre d'arbres. Plus il est élevé, plus le modèle est stable (mais plus lent à entraîner).                       |
| `max_depth`         | 15              | Profondeur maximale des arbres. Limite le surapprentissage.                                                       |
| `min_samples_split` | 10              | Nombre min d’échantillons requis pour diviser un nœud. Contrôle la complexité de l’arbre.                         |
| `min_samples_leaf`  | 5               | Nombre min d’échantillons requis dans une feuille terminale. Plus il est grand, plus l'arbre est régularisé.      |
| `max_features`      | 0.5             | Proportion de variables à considérer pour chaque split. Ici, 50 % des variables sont utilisées à chaque division. |


## **Réentraîner le RandomForestRegressor avec les meilleurs hyperparamètres**

In [10]:
# Modèle avec les meilleurs hyperparamètres
best_model= Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('regressor', RandomForestRegressor(
                                        n_estimators=200,
                                        max_depth=15,
                                        max_features=0.5,
                                        min_samples_split=10,
                                        min_samples_leaf=5,
                                        random_state=42,
                                        n_jobs=-1
    ))
])


# Entraînement
best_model.fit(X_train, y_train)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_col',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['profil_cotier', 'saison',
                                                   'city'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=15, max_features=0.5,
                                       min_samples_leaf=5, min_samples_split=10,
                                       n_estimators=200, n_jobs=-1,
                                       random_state=42))])

In [11]:
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)


In [12]:
print(f"R² Train :{ r2_score(y_train, y_pred_train):.4f}")
print(f"R² Test  :{ r2_score(y_test, y_pred_test):.4f}")
print(f"MSE Test : {mean_squared_error(y_test, y_pred_test):.4f}")

R² Train :0.8692
R² Test  :0.8448
MSE Test : 1.7450


#### Ces performances montrent que le modèle parvient à **expliquer environ 87 % de la variance** dans les données d'entraînement, tout en conservant une capacité de généralisation élevée avec **84 % de variance expliquée sur les données de test**. L'écart entre les deux scores est faible (environ 3 %), ce qui est **un excellent indicateur de stabilité**.

> Un écart faible entre R² train et test indique que le modèle n'est **pas en situation de surapprentissage** (overfitting). Il a appris les relations sous-jacentes dans les données sans les mémoriser.

In [13]:
import numpy as np
# 1. Convertir y_pred en DataFrame si besoin
if isinstance(y_pred_test, np.ndarray):
    y_pred_test = pd.DataFrame(y_pred_test, columns=y_test.columns)

# 2. Concaténer les données
df_long = pd.DataFrame()
for col in y_test.columns:
    temp_df = pd.DataFrame({
        'product': [col] * len(y_test),
        'y_true': y_test[col].values,
        'y_pred': y_pred_test[col].values
    })
    df_long = pd.concat([df_long, temp_df], ignore_index=True)

# 3. Calculer les métriques
metrics = []
for col in y_test.columns:
    r2 = r2_score(y_test[col], y_pred_test[col])
    mse = mean_squared_error(y_test[col], y_pred_test[col])
    metrics.append({'product': col, 'R2': r2, 'MSE': mse})

df_metrics = pd.DataFrame(metrics)

# 4. Fusion finale
df_final = df_long.merge(df_metrics, on='product')


df_summary = df_final.groupby('product')[['R2', 'MSE']].first().reset_index()
print(df_summary)


                     product        R2       MSE
0                    Charbon  0.886464  0.747981
1           boissons chaudes  0.953461  1.479488
2          boissons fraiches  0.954411  0.887371
3              crème solaire  0.910759  0.633270
4   produits anti_moustiques  0.897105  2.830542
5      produits de jardinage  0.862513  0.601709
6    produits laitiers frais  0.794283  3.111544
7               snacks salés  0.607178  2.724019
8              snacks sucrés  0.752617  2.938428
9           soins hydratants  0.893305  2.323133
10             soins hygiene  0.798545  0.590468
11       ustensiles jetables  0.963975  1.989946
12     équipements d urgence  0.707222  1.827152




En comparant les performances des deux modèles (**DecisionTreeRegressor** vs **RandomForestRegressor**) sur différentes catégories de produits, nous constatons les éléments suivants :

### Améliorations avec RandomForestRegressor

Certaines catégories affichent une **amélioration significative** du score R², indiquant une meilleure capacité du modèle à expliquer la variance des ventes :

* **Équipements d’urgence** :

  * `R²` passe de **0.288 → 0.332**, soit une amélioration de \~15%.
  * MSE réduit : **1.31 → 1.23**

* **Soins hydratants** :

  * `R²` passe de **0.242 → 0.303**, soit une amélioration de \~25%.
  * MSE réduit : **0.83 → 0.76**

Ces produits sont généralement **moins fréquents** ou ont des comportements de vente **plus complexes**, ce qui explique pourquoi un modèle plus robuste comme la **forêt aléatoire** (Random Forest) arrive à mieux capturer leurs schémas, malgré des scores R² encore faibles.

---

### **Analyse contextuelle des faibles scores R²**

#### 1. **Équipements d’urgence**

* **Hypothèse logique** : Ces produits sont **fortement corrélés à des événements rares**, comme les **tempêtes** ou **pluies intenses** pouvant causer des coupures d'électricité.
* **Variables pertinentes** : `indice_tempete`, `indice_pluie_intense` (binaires 0/1).


### 2. **Soins hydratants**

* **Hypothèse logique** : Les ventes augmentent en **hiver** et en cas de **sécheresse**.
* **Variables pertinentes** : `indice_secheresse` (0/1), `saison_hiver`


---



In [14]:
print(df['Indice de Tempête'].value_counts())
print(df['Indice de Pluie Intense'].value_counts())
print(df['Indice de sécheresse'].value_counts())


Indice de Tempête
0    43433
1      336
Name: count, dtype: int64
Indice de Pluie Intense
0    41441
1     2328
Name: count, dtype: int64
Indice de sécheresse
0    31415
1    12354
Name: count, dtype: int64


| Indice                      | `0`    | `1`    | % d'occurrence `1` | Problème ?          |
| --------------------------- | ------ | ------ | ------------------ | ------------------- |
| **Indice de Tempête**       | 43 433 | **336**  | **0.7%**             | oui- presque inutilisable |
| **Indice de Pluie Intense** | 41 441 | 2 328  | **5.3%**           | Faible            |
| **Indice de Sécheresse**    | 31 415 | 12 354 | **28.2%**          | ça marche                 |
---


#### 1. **`Indice de Tempête` est inexploitable**

* Il est **toujours à 0** : le modèle **ne peut rien apprendre** de cette feature.


#### 2. **`Indice de Pluie Intense` est faiblement présent**

* 5% d’occurrence, ce n’est **pas dramatique** mais c’est **trop peu pour bien modéliser son effet**.
*  À risque d’être ignoré par RandomForest.

#### 3. **`Indice de Sécheresse` est bien distribué**

* 28% de `1` ⇒ c’est exploitable 

---


 # **Solution :** 
 ## **Suréchantillonner les jours avec `Indice de Pluie Intense = 1`**
 ### **Pourquoi suréchantillonner `Indice de Pluie Intense = 1 `?**
>**le modèle apprend surtout à prédire les cas "normaux" (pas de pluie intense), car ils sont écrasants en quantité.**
>**il n’apprend presque rien des jours où il y a de fortes pluies, donc il ne sait pas bien adapter les prévisions de ventes d'équipements d'urgence pendant ces jours-là**

## **Suréchantillonnage : c’est quoi ?**
**L’idée est de créer artificiellement plus de jours rares (avec Indice de Pluie Intense = 1) pour que le modèle les prenne au sérieux pendant l’entraînement.**
**On fait ça en dupliquant aléatoirement ces lignes rares. Cela ne change pas la vérité des données, mais équilibre la vision du modèle.**

#### on a créé `Indice de Pluie Intense = 1 `si `precipitation > 50`.
#### Donc, cette variable binaire est dérivée directement d’une variable continue (precipitation). Si on duplique les lignes où Indice = 1, on duplique aussi les mêmes précipitations extrêmes.
### **même travail avec `Indice de Tempête = 1 `si `vitesse_vent > 25m/s`**
### **Problème potentiel** :  
#### **le modèle peut apprendre à associer exactement les mêmes valeurs de précipitation à certaines ventes, ce qui réduit la diversité dans l’espace des cas rares.**

## **Pourquoi ce n’est pas vraiment une falsification ?**
#### **On ne modifie pas les données, on rééquilibre la fréquence d’un événement dans le jeu d'entraînement. On ne fais pas dire aux données quelque chose de faux, on leur dis juste : “apprends mieux ce type de jour“**
##  Option  Possibles : 
| Option | Méthode                          | Description                                                                 | Avantages                                                                 | Inconvénients                                                              |
|--------|----------------------------------|-----------------------------------------------------------------------------|---------------------------------------------------------------------------|----------------------------------------------------------------------------|
| **1**  | **Suréchantillonnage + bruit (jittering)** | Ajout d'un léger bruit aléatoire aux variables continues pour éviter les duplications exactes. | - Simple à implémenter<br>- Préserve la distribution globale des données | - Peut introduire du bruit artificiel<br>- Risque de sur-ajustement si mal calibré |
| **2**  | **Augmentation synthétique (SMOTE/GANs)**  | Génération de données synthétiques via :<br>- SMOTE pour régression<br>- Modèles génératifs (VAEs, GANs) | - Crée des variations réalistes<br>- Utile pour les petits jeux de données | - Complexe à mettre en œuvre<br>- SMOTE peu adapté aux problèmes de régression<br>- Coût computationnel élevé (GANs) |
| **3**  | **Pondération**                  | Attribution de poids plus élevés aux échantillons rares pendant l'entraînement. | - Aucune modification des données<br>- Pris en charge par XGBoost/LightGBM | - Limité aux modèles supportant les poids<br>- Moins efficace si déséquilibre extrême |

##  **Pourquoi il faut faire l'augmentation uniquement sur X_train (l’ensemble d'entraînement)?**
| Si on  le fait **sur tout le dataset (train + test)**                                                                    | Si on le fait **uniquement sur `X_train`**                                                                                |
| ------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------- |
| On risque de **polluer le jeu de test avec des données artificielles**, donc de fausser la mesure réelle de performance. | On **entraîne le modèle sur un jeu enrichi**, mais on le teste sur des données **restées 100% naturelles et réalistes**. |
| Cela donne un **R²,  MSE trop optimistes**, car on teste sur des données similaires à celles générées.             | On a une **évaluation honnête de la généralisation du modèle**, ce qui est crucial en machine learning.                    |


In [15]:
import numpy as np
# ---------------------
# Indice de Tempête
# ---------------------
tempete_1 = X_train[X_train['Indice de Tempête'] == 1].copy()
y_tempete_1 = y_train[X_train['Indice de Tempête'] == 1].copy()

n_tempete_to_add = 5000

aug_tempete = tempete_1.sample(n=n_tempete_to_add, replace=True, random_state=42).copy()
y_aug_tempete = y_tempete_1.sample(n=n_tempete_to_add, replace=True, random_state=42).copy()

# Ajout d'un léger bruit sur la vitesse du  (+1.5 -1.5)
if 'vitesse_vent' in aug_tempete.columns:
    aug_tempete['vitesse_vent'] += np.random.normal(0, 1.5, size=n_tempete_to_add)
    aug_tempete['vitesse_vent'] = aug_tempete['vitesse_vent'].clip(lower=0)
    aug_tempete['Indice de Tempête'] = (aug_tempete['vitesse_vent'] > 25).astype(int)

# ---------------------
# Indice de Pluie Intense
# ---------------------
pluie_1 = X_train[X_train['Indice de Pluie Intense'] == 1].copy()
y_pluie_1 = y_train[X_train['Indice de Pluie Intense'] == 1].copy()

n_pluie_to_add = 5000
aug_pluie = pluie_1.sample(n=n_pluie_to_add, replace=True, random_state=42).copy()
y_aug_pluie = y_pluie_1.sample(n=n_pluie_to_add, replace=True, random_state=42).copy()
# Ajout d'un léger bruit sur la vitesse du vent (+2 -2)

aug_pluie['precipitation'] += np.random.normal(0, 2, size=n_pluie_to_add)
aug_pluie['precipitation'] = aug_pluie['precipitation'].clip(lower=0).astype(int)
aug_pluie['Indice de Pluie Intense'] = (aug_pluie['precipitation'] > 50).astype(int)

# ---------------------
# Fusion finale
# ---------------------
X_train_aug2 = pd.concat([X_train, aug_tempete, aug_pluie], ignore_index=True)
y_train_aug2 = pd.concat([y_train, y_aug_tempete, y_aug_pluie], ignore_index=True)

# Vérification
print("Taille initiale :", len(X_train))
print("Taille après augmentation :", len(X_train_aug2))
print("\nNouvelle distribution :")
print(X_train_aug2['Indice de Tempête'].value_counts())
print(X_train_aug2['Indice de Pluie Intense'].value_counts())


Taille initiale : 35015
Taille après augmentation : 45015

Nouvelle distribution :
Indice de Tempête
0    41404
1     3611
Name: count, dtype: int64
Indice de Pluie Intense
0    36556
1     8459
Name: count, dtype: int64


In [16]:
# Pipeline complet
model_aug = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('regressor', RandomForestRegressor(
                                        n_estimators=150,
                                        max_depth=10,
                                        max_features=0.5,
                                        min_samples_split=10,
                                        min_samples_leaf=5,
                                        random_state=42,
                                        n_jobs=-1
    ))
])

# Entraînement
model_aug.fit(X_train_aug2, y_train_aug2)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_col',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['profil_cotier', 'saison',
                                                   'city'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=10, max_features=0.5,
                                       min_samples_leaf=5, min_samples_split=10,
                                       n_estimators=150, n_jobs=-1,
                                       random_state=42))])

In [18]:
#prediction :
y_pred_train_2 = model_aug.predict(X_train_aug2)
y_pred_test_2 = model_aug.predict(X_test)


In [19]:
print(f"R² Train :{ r2_score(y_train_aug2, y_pred_train_2):.4f}")
print(f"R² Test  :{ r2_score(y_test, y_pred_test_2):.4f}")
print(f"MSE Test : {mean_squared_error(y_test, y_pred_test_2):.4f}")

R² Train :0.8595
R² Test  :0.8170
MSE Test : 2.1018



---

## Ce que on a fait :

| Étape                   | Détail                                                                                                                           |
| ----------------------- | -------------------------------------------------------------------------------------------------------------------------------- |
|  Objectif             | Faire apprendre au modèle à mieux prédire les jours rares (pluie intense) pour mieux prévoir les ventes d’équipements d'urgence. |
|  Définition jour rare | `Indice de Pluie Intense = 1` si `precipitation > 50` ,`Indice de Tempête = 1 `si `vitesse_vent > 25m/s`                                                                            |
|  Méthode              | Dupliquer les lignes rares (avec `Indice = 1`) pour équilibrer leur fréquence.                                                   |
|  Jittering            | on ajoute un bruit léger sur les variables numériques pour ne pas avoir des copies exactes.                                     |
---

## Pourquoi ça ne marche pas bien ?

1. ###  **Corrélation artificielle entre météo et vente** :

   Même avec du jittering, le modèle **voit plusieurs fois le même scénario météo avec (presque) les mêmes ventes** → il apprend à “mémoriser” ce cas plutôt que généraliser.

2. ###  **Pas de diversité dans les cas extrêmes** :

   Les jours de pluie intense sont **rarement identiques dans la réalité**. Il y a :

   * des différences de température,
   * de saison (été vs hiver),
   * d’humidité, de vent, de jour de la semaine...
     On n’a pas introduit cette **variabilité contextuelle**, donc le modèle **sur-apprend quelques cas rares très spécifiques** → pas généralisable.

3. ###  **Bruit ≠ Variété structurelle** :

   Le jittering reste **local** (gaussien), donc il **n’élargit pas réellement l’espace des cas rares**, il ne fait que des variantes très proches.

---


# **augmentation métier**
---

In [20]:
# Etape 1 : Extraire les cas rares : fortes pluies OU tempêtes
X_rares = X_train[(X_train['Indice de Pluie Intense'] == 1) | (X_train['Indice de Tempête'] == 1)]
print("Nombre de cas rares :", X_rares.shape[0])


Nombre de cas rares : 2032


### **Générer des cas météo rares plausibles (Data Augmentation)**
>  créer de nouvelles lignes météo plausibles en se basant sur les cas réels rares (pluie intense ou tempête), en ajoutant un léger bruit contrôlé (jitter), et sans casser les corrélations existantes.

In [21]:
#Etape 2 : Génération de cas synthétiques
# Définir combien de fois on veut dupliquer chaque ligne rare (par exemple : 3 fois)
N_DUP = 3

# Cas rares météo (pluie intense ou tempête)

# Duplication
X_train_aug = pd.concat([X_rares] * N_DUP, ignore_index=True)

In [22]:
def jitter_group(data, group_cols, std_fraction=0.1, seed=None):
    if seed is not None:
        np.random.seed(seed)
    data = data.copy()
    std = data[group_cols[0]].std()
    noise = np.random.normal(0, std_fraction * std, size=len(data))
    for col in group_cols:
        data[col] += noise
    return data

# Application cohérente
X_train_aug = jitter_group(X_train_aug, ['température', 'temp_lag1', 'temp_lag3', 'temp_lag7'], std_fraction=0.1)
X_train_aug = jitter_group(X_train_aug, ['humidity', 'humid_lag1', 'humid_lag3', 'humid_lag7'], std_fraction=0.1)
X_train_aug = jitter_group(X_train_aug, ['vitesse_vent', 'vitesse_vent_lag1', 'vitesse_vent_lag3', 'vitesse_vent_lag7'], std_fraction=0.1)
X_train_aug = jitter_group(X_train_aug, ['precipitation', 'precip_lag1', 'precip_lag3', 'precip_lag7'], std_fraction=0.1)


In [23]:
#Recalculer les indices extrêmes (à partir des nouvelles valeurs)
X_train_aug['Indice de Pluie Intense'] = (X_train_aug['precipitation'] >= 50).astype(int)
X_train_aug['Indice de Tempête'] = (X_train_aug['vitesse_vent'] >= 25).astype(int)


### **Générer les ventes synthétiques pour les produits ciblés**

In [24]:


# 1. Sélection des indices des cas extrêmes (pluie intense ou tempête)
indices_extremes = y_train[(X_train['Indice de Pluie Intense'] == 1) | 
                           (X_train['Indice de Tempête'] == 1)].index  

n_aug = len(X_train_aug)

# 2. Échantillonnage aléatoire de lignes réelles de y_train comme base
y_train_base = y_train.loc[np.random.choice(indices_extremes, size=n_aug, replace=True)].copy()

# 3. Ajout d'une variation de +5% avec du bruit normal
for col in ['équipements d urgence']:
    y_train_base[col] *= np.random.normal(loc=1.05, scale=0.05, size=n_aug)
    y_train_base[col] = y_train_base[col].clip(lower=0)  

# 4. Fusion des données originales et augmentées
X_train_final = pd.concat([X_train, X_train_aug], ignore_index=True)
y_train_final = pd.concat([y_train, y_train_base], ignore_index=True)


In [25]:
model= Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('regressor', RandomForestRegressor(
                                        n_estimators=200,
                                        max_depth=20,
                                        max_features=None,
                                        min_samples_split=6,
                                        min_samples_leaf=3,
                                        random_state=42,
                                        n_jobs=-1
    ))
])

model.fit(X_train_final,y_train_final)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_col',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['profil_cotier', 'saison',
                                                   'city'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=20, max_features=None,
                                       min_samples_leaf=3, min_samples_split=6,
                                       n_estimators=200, n_jobs=-1,
                                       random_state=42))])

In [26]:
#prediction :
y_pred_train_3 = model.predict(X_train_final)
y_pred_test_3 = model.predict(X_test)


In [27]:
print(f"R² Train :{ r2_score(y_train_final, y_pred_train_3):.4f}")
print(f"R² Test  :{ r2_score(y_test, y_pred_test_3):.4f}")
print(f"MSE Test : {mean_squared_error(y_test, y_pred_test_3):.4f}")

R² Train :0.8548
R² Test  :0.8428
MSE Test : 1.7528


In [28]:
import numpy as np
# 1. Convertir y_pred en DataFrame si besoin
if isinstance(y_pred_test_3, np.ndarray):
    y_pred_test_3 = pd.DataFrame(y_pred_test_3, columns=y_test.columns)

# 2. Concaténer les données
df_long = pd.DataFrame()
for col in y_test.columns:
    temp_df = pd.DataFrame({
        'product': [col] * len(y_test),
        'y_true': y_test[col].values,
        'y_pred': y_pred_test_3[col].values
    })
    df_long = pd.concat([df_long, temp_df], ignore_index=True)

# 3. Calculer les métriques
metrics = []
for col in y_test.columns:
    r2 = r2_score(y_test[col], y_pred_test_3[col])
    mse = mean_squared_error(y_test[col], y_pred_test_3[col])
    metrics.append({'product': col, 'R2': r2, 'MSE': mse})

df_metrics = pd.DataFrame(metrics)

# 4. Fusion finale
df_final = df_long.merge(df_metrics, on='product')


df_summary = df_final.groupby('product')[['R2', 'MSE']].first().reset_index()
print(df_summary)


                     product        R2       MSE
0                    Charbon  0.876591  0.813027
1           boissons chaudes  0.958674  1.313770
2          boissons fraiches  0.956117  0.854168
3              crème solaire  0.906605  0.662751
4   produits anti_moustiques  0.903649  2.650524
5      produits de jardinage  0.882481  0.514321
6    produits laitiers frais  0.787078  3.220537
7               snacks salés  0.591744  2.831045
8              snacks sucrés  0.743257  3.049608
9           soins hydratants  0.888418  2.429542
10             soins hygiene  0.794848  0.601303
11       ustensiles jetables  0.963988  1.989187
12     équipements d urgence  0.702463  1.856855


---

###  **Résultats comparatifs des approches d'entraînement**

| Méthode d'entraînement  | R² Train | R² Test | MSE Test     |
| ----------------------- | -------- | ------- | ------------ |
| **Sans augmentation**   | 0.87   | 0.84 | **1.74**  |
| **Augmentation métier** | 0.85   | 0.84  | 1.75       |
| **Bruit + jittering**   | 0.86   | 0.81  | 2.1      |



*  **Meilleure performance globale** (sur le test set) : **Modèle sans augmentation**.
*  **L’augmentation avec bruit + jittering** dégrade **visiblement** les performances.
*  **L’augmentation métier** (cas extrêmes + ajustement des ventes) donne **des résultats proches** du modèle de base, avec un léger écart de performance globale.

---



In [29]:
indices_extremes = X_test[
    (X_test['Indice de Pluie Intense'] == 1) | 
    (X_test['Indice de Tempête'] == 1)
].index

# Pour chaque modèle : calcul des erreurs sur ces jours-là
y_test_ext = y_test.loc[indices_extremes]

# Pour le modèle de base :
y_pred_base_ext = best_model.predict(X_test.loc[indices_extremes])
mse_base_ext = mean_squared_error(y_test_ext, y_pred_base_ext)
r2_base_ext = r2_score(y_test_ext, y_pred_base_ext)

# Idem pour le modèle métier :
y_pred_met_ext = model.predict(X_test.loc[indices_extremes])
mse_met_ext = mean_squared_error(y_test_ext, y_pred_met_ext)
r2_met_ext = r2_score(y_test_ext, y_pred_met_ext)

# Idem pour le modèle bruité :
y_pred_jitter_ext = model_aug.predict(X_test.loc[indices_extremes])
mse_jitter_ext = mean_squared_error(y_test_ext, y_pred_jitter_ext)
r2_jitter_ext = r2_score(y_test_ext, y_pred_jitter_ext)


In [30]:
results = pd.DataFrame({
    'Modèle': ['Sans augmentation', 'Augmentation métier', 'Jittering'],
    'R² (cas extrêmes)': [r2_base_ext, r2_met_ext, r2_jitter_ext],
    'MSE (cas extrêmes)': [mse_base_ext, mse_met_ext, mse_jitter_ext]
})

print(results)


                Modèle  R² (cas extrêmes)  MSE (cas extrêmes)
0    Sans augmentation           0.755095            1.967374
1  Augmentation métier           0.727556            2.282693
2            Jittering           0.739682            2.063144



---

###  **Interprétation**

1. **Modèle sans augmentation** reste le **meilleur** sur les cas extrêmes :

   * Il a le **plus haut R²** (donc meilleure capacité explicative).
   * Il a aussi le **plus bas MSE**, donc les erreurs de prédiction sont plus faibles.

2. **Augmentation métier (pluie/tempête simulées)** :

   * Elle **dégrade la performance** sur les cas extrêmes.
   
3. **Jittering** fait **un peu mieux** que l'augmentation métier, mais reste **inférieur au modèle original**.

---

